In [5]:
struct Dual{T} <: Number
    v::T
    dv::Vector{T}
end


In [52]:
import Base: +, -, *, /
-(x::Dual) = Dual(-x.v,-x.dv)
+(x::Dual, y::Dual) = Dual( x.v + y.v, x.dv + y.dv)
-(x::Dual, y::Dual) = Dual( x.v - y.v, x.dv - y.dv)
*(x::Dual, y::Dual) = Dual( x.v * y.v, x.dv * y.v + x.v * y.dv)
*(x::Dual, y::Number) = Dual( x.v * y, x.dv * y)
/(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)
/(x::Dual, y::Number) = Dual( x.v / y, x.dv/y)

import Base: abs, sin, cos, tan, exp, sqrt, isless
abs(x::Dual) = Dual(abs(x.v),sign(x.v)*x.dv)
sin(x::Dual) = Dual(sin(x.v), cos(x.v)*x.dv)
cos(x::Dual) = Dual(cos(x.v),-sin(x.v)*x.dv)
tan(x::Dual) = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
exp(x::Dual) = Dual(exp(x.v), exp(x.v)*x.dv)
sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
isless(x::Dual, y::Dual) = x.v < y.v;

import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

import Base: convert, promote_rule

convert(::Type{Dual{T}}, x::Dual{T}) where T =
Dual(convert(T, x.v), convert(Vector{T}, x.dv))

convert(::Type{Dual{T}}, x::Number) where T =
Dual(convert(T, x), Vector(0))

promote_rule(::Type{Dual{T}}, ::R) where {T,R} =
Dual{promote_type(T,R)}

promote_rule (generic function with 123 methods)

In [53]:
a = Dual(2, [1, 0, 0])
b = Dual(3, [0, 1, 0])
c = Dual(10, [0, 0, 1])
println(c)
println(typeof(c))

(10) + [[0, 0, 1]ϵ]
Dual{Int64}


In [56]:
a+b

(5) + [[1, 1, 0]ϵ]

In [21]:
function test_f(y, x, z)
    y^3 + x^2 - 2z;
end

test_f (generic function with 1 method)

In [21]:
test_f

test_f (generic function with 1 method)

In [22]:
test_f(a,b,c)

(7) + [[12, 6, -1]ϵ]

In [17]:
wektor = [2,3,10]

3-element Array{Int64,1}:
  2
  3
 10

In [18]:
ceil(2.1)

3.0

In [19]:
J = function jacobian(f, chunk_leng, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix[]
    x = Dual{1, T}[]
    println(x)
    for i=1:length(ceil(length(args)/chunk_leng))
        for j=1:length(args)
            seed = (j > (i-1) * chunk_leng && j <= i*chunk_leng)
            if seed
                println("tutaj2")
                oney = zeros(Int64, chunk_leng)
                index = mod(j -1, chunk_leng)
                oney[index + 1] = 1
                println(oney)
                println(typeof(Dual(args[j], oney)))
                push!(x, Dual(args[j], oney))
                println("tutaj")
            else
                println("tutaj3")
                push!(x, Dual(args[j], zeros(Int64, chunk_leng)))
            end
            println("wewnetrzny for")
        end
        println(x)
        column = partials.([f(x...)...])
        println(column)
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [20]:
j = J(test_f, 2, wektor)

ErrorException: [91mtoo many parameters for type[39m